EpistemicPropagationBlackBboxWorkshop

# Import the needed libraries

In [2]:
import subprocess  # needed for running the function
import os  # needed for running the function
#from PyUncertainNumber.UC.uncertainNumber import UncertainNumber
#from PyUncertainNumber.UC.utils import UNEncoder, cd_root_dir
import numpy as np

# XFOIL

XFOIL is an interactive program for analyzing isolated subsonic airfoils, available under the GNU General Purpose License. For more details, see https://web.mit.edu/drela/Public/web/xfoil .

A special version, v6.99uq, has been compiled with enhanced features like increased output precision, a higher maximum panel count (2048), and adjustable boundary layer convergence criteria. Executables for Windows, MacOS, and Linux, provided by Marshall Galbraith of MIT, are available in a compressed tar file found in the link below

https://acdl.mit.edu/galbramc/UQ_Challenge/xfoil_execs.tgz


# Problem setup

Use XFOIL to compute the **lift**, c_l, and **moment** c_m, **coefficients** for:

- NACA 2412 airfoil
- simple flap starting at 70% chord and hinged at the midpoint
- incompressible, viscous flow
- 100 panels are to be used in all analyses except in estimating the discretisation error

Assess whether, at nominally zero degree angle of attack and flap deflection:
-  0.155 ≤ c_l ≤  0.265 
- -0.050 ≤ c_m ≤ -0.044

##TODO what can we say about their dependence/correlation? 
##TODO the software shoudl give you the input combinations, without a function.
##TODO endpoint cauchy, bernstein ??
##TODO only with aleatory , just monte carlo, what about dependence/correlation? 
##TODO Mitcell truss, how it would work given there is not intrusive access to the code or possibility to run it through python. 

# Problem parameterisation
Table 1. Summary of uncertain parameters 
![alt text](<Screenshot 2024-11-05 155042.png>)

# Create the function

A function is created which calculates the lift coefficient (c_l) and moment coefficient (c_m) for a given set of input parameters using XFOIL
- The function's input is only the uncertain numbers.
- The deterministic inpyt are defined within the function. 
- As sampling techniques are used it is likley that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this. 

In [52]:
def coef_xfoil(x):
    """
    Calculates the lift coefficient (c_l) and moment coefficient (c_m) for a given 
    set of input parameters using XFOIL.

    Args:
        x np.array: An np.array of input parameters containing:
            x[0]: Flap deflection angle (float)
            x[1]: Reynolds number (float)
            x[2]: Upper surface trip location (float)
            x[3]: Lower surface trip location (float)
            x[4]: Angle of attack (alpha) (float)

    Returns:
        np.array: A numpy array containing the lift coefficient (c_l) and moment coefficient (c_m).
    """
    try:
        # Certain Input parameters
        airfoil_name = 2412
        pacc = 1
        flap_hinge_x = 0.70  
        flap_hinge_y = 999
        flap_hinge_r = 0.50
        no_panel_nodes = 256
        Mach = 0.00 
       
        # Create the XFOIL input file
        input_file = open("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\input_file_cha.in", 'w')
        input_file.write("NACA {0}\n".format(airfoil_name))
        input_file.write("gdes\n")
        input_file.write("flap\n")
        input_file.write(str(flap_hinge_x))
        input_file.write("\n")
        input_file.write(str(flap_hinge_y))
        input_file.write("\n")
        input_file.write(str(flap_hinge_r))
        input_file.write("\n")
        input_file.write(str(format(x[0], '.4f')))  # flap deflection
        input_file.write("\n")
        input_file.write("exec\n\n")
        input_file.write("ppar\n")
        input_file.write("n {0}\n".format(no_panel_nodes))
        input_file.write("p {0}\n".format(1.0))
        input_file.write("\n\n")
        input_file.write("oper\n")
        input_file.write("mach {0}\n".format(Mach))
        #input_file.write("iter {0}\n".format(100))
        input_file.write("visc\n")
        input_file.write(str(format(x[1], '.0f')))  # Reynolds number
        input_file.write("\n")
        input_file.write("vpar\n")
        input_file.write("xtr\n")
        input_file.write(str(format(x[2], '.4f')) ) # Upper Surface Trip Location
        input_file.write("\n")
        input_file.write(str(format(x[3], '.4f'))) # Lower Surface Trip Location
        input_file.write("\n\n")
        input_file.write("pacc {0}\n".format(pacc))
        input_file.write("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up.txt\n\n")
        input_file.write("alfa\n")
        input_file.write(str(format(x[4], '.4f')))  # alpha
        input_file.write("\n\n")
        input_file.write("quit\n")
        input_file.close()
        
        # Run XFOIL
        subprocess.call("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\xfoil.exe < C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\input_file_cha.in", shell=True)
        
        # Extract results from the output file
        old_file_name = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up"  # Or the current file name with any extension
        new_file_name = "C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up.txt"
        os.rename(old_file_name, new_file_name)

        results = np.loadtxt(("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up.txt"), 
                     skiprows=12, encoding='latin1')
        
        c_l = results[1]  # Extract lift coefficient
        c_m = results[4]  # Extract moment coefficient

    except FileNotFoundError:
        print("Error: XFOIL executable not found. Please check the path.")
        return np.array([0.0, 0.0])
    except IndexError:
        print("Error: XFOIL output file is empty or has unexpected format.")
        return np.array([0.0, 0.0])
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return np.array([0.0, 0.0])
    finally:        
        # Remove the output file (if it exists)
        try:
            os.unlink("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up.txt")
        except FileNotFoundError:
            pass

    # Return the results as a numpy array
    return np.array([c_l, c_m]) 


#### Test the installation

To ensure that the installation is correct, try to match the results with the following expected output values. Variations in the final 3-4 significant figures are expected. 

In [37]:
# test the function

x = np.array([0.5, 500000, 0.3, 0.7, 0])

y = coef_xfoil(x)

# Comment out the following lines from the coef_xfoil function so the output file is not deleted after each run.
#finally:
        # Remove the output file (if it exists)
      #  try:
           # os.unlink("C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\xfoil_workshop\\output_up.txt")
     #   except FileNotFoundError:
      #      pass

# Open the output.txt file and check that the following coeffcients 

#CL      = 0.245832312525558E+00
#CD      = 0.897255056288259E-02
#CDp     = 0.639220543894256E-03    
#CM      =-0.525018720205690E-01
#Top_Xtr = 0.300000000000043E+00 
#Bot_Xtr = 0.700000000000001E+00 
#Top_Itr = 0.286192798167708E+02 
#Bot_Itr = 0.881204554341905E+02

## TODO DO not forget to uncomment the lines when propagating the uncertainty in the input parameters. 

# Propagate aleatory uncertainty 

Step 2. Create 5 uncertain numbers assuming they follow a normal distribution with the parameters provided in Table 1.

In [45]:
#from PyUncertainNumber.UC.uncertainNumber import 
import importlib
import PyUncertainNumber.UP.endpoint as up


importlib.reload(up)

<module 'PyUncertainNumber.UP.endpoint' from 'C:\\Users\\Ioanna\\Documents\\GitHub\\daws2\\src\\PyUncertainNumber\\UP\\endpoint.py'>

# Propagate epistemic uncertainty 

Create 5 uncertain numbers assuming htey are intervals with the lower and upper values provided in Table 1. 

In [46]:
# %% Inputs
#Define input intervals
flap_deflection = np.array([-0.24,0.24]) 
Re = np.array([492500, 507500])
xtr_top = np.array([0.255, 0.345])
xtr_bottom = np.array([0.637, 0.763])
alpha = np.array([-0.3, 0.3])

# Create a 2D np.array with all uncertain input parameters in the **correct** order.
xInt = np.array([flap_deflection, Re,  xtr_top, xtr_bottom, alpha])
print(xInt)

[[-2.400e-01  2.400e-01]
 [ 4.925e+05  5.075e+05]
 [ 2.550e-01  3.450e-01]
 [ 6.370e-01  7.630e-01]
 [-3.000e-01  3.000e-01]]


- Step 3. Choose multiple black box techniques to propagate the intervals through the model. 

It should be noted that for this example it makes sense to use...

### Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of n = 2**d. 

For the working example, there are d = 5 intervals which results in n = 2**5 = 32 input combinations.

#### Assumptions




In [42]:
# How many input combinations are expected from the endpoint propagation?
d = 5 # is the number of uncertain input expressed as intervals.
n = 2**d # The total number of combinations 
print("Total number of input combinations for the endpoint method:", n) 

Total number of input combinations for the endpoint method: 32


The user has 

In [53]:
min_y, max_y, x_miny, x_maxy = up.endpoints_method(xInt, coef_xfoil, save_raw_data = 'no')



<class 'numpy.ndarray'>


100%|██████████| 31/31 [00:06<00:00,  4.95it/s]


In [ ]:
# check the results.
print(min_y)
print(max_y)
print(x_miny)
print(x_maxy)

[ 0.1523357  -0.05083742]
[ 0.26711539 -0.0434172 ]
[[-2.400e-01  4.925e+05  2.550e-01  7.630e-01 -3.000e-01]
 [ 2.400e-01  5.075e+05  3.450e-01  6.370e-01  3.000e-01]]
[[ 2.400e-01  5.075e+05  3.450e-01  6.370e-01  3.000e-01]
 [-2.400e-01  4.925e+05  2.550e-01  7.630e-01 -3.000e-01]]


### Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [27]:
# How many input combinations are expected from the subinterval reconstitution propagation?
d = 5 # is the number of uncertain input expressed as intervals.
m = 4 # is the number of partitions. 
n = (m+1)**d # The total number of combinations 
print("Total number of input combinations for the subinterval reconstitution method:", n) 

Total number of input combinations for the subinterval reconstitution method: 3125


### Sampling propagation